In [ ]:
import cv2
import glob
import numpy as np
from skimage import img_as_ubyte
import matplotlib.pyplot as plt
import os
import pandas as pd

# Pfad zu des ordners einfügen
path = "..."

files = sorted(glob.glob(os.path.join(path, "*.jpg")))


results = []

for img_number, file in enumerate(files):
    bild = cv2.imread(file)
    if bild is None:
        continue

    bild2 = bild

    a, b = bild2.shape[:2]

    if a > b:
        bild2 = cv2.rotate(bild2, cv2.ROTATE_90_CLOCKWISE)

    plt.imshow(cv2.cvtColor(bild2, cv2.COLOR_BGR2RGB))
    bild = bild2
    plt.show()

    bild3 = bild2[738:1678, 2726:5000]
    bild2 = bild3
    hsv = cv2.cvtColor(bild2, cv2.COLOR_BGR2HSV)
    # maske rot
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([179, 255, 255])
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    # kombination maske

    mask = cv2.bitwise_or(mask1, mask2)
    rotallein = cv2.bitwise_and(bild2, bild2, mask=mask)
    nurrot_rgb = cv2.cvtColor(rotallein, cv2.COLOR_BGR2RGB)

    grau = cv2.cvtColor(nurrot_rgb, cv2.COLOR_BGR2GRAY)

    contours, _ = cv2.findContours(grau, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Neue leere Maske
    maskeneu = np.zeros_like(grau)
    # Flecken entfernen
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:  # Threshold
            cv2.drawContours(maskeneu, [cnt], -1, 255, thickness=cv2.FILLED)
    plt.imshow(maskeneu)
    linie = cv2.Canny(maskeneu, 80, 90)  # Canny

    weiss = np.argwhere(linie == 255)  # weisse pixel
    weiss_x = weiss[:, 1]
    weiss_y = -weiss[:, 0]

    # ab hier geändert
    if weiss.size == 0:
        bild = cv2.rotate(bild, cv2.ROTATE_180)
        bild3 = bild[738:1678, 2726:5000]
        bild2 = bild3
        hsv = cv2.cvtColor(bild2, cv2.COLOR_BGR2HSV)
        # maske rot
        lower_red1 = np.array([0, 100, 100])
        upper_red1 = np.array([10, 255, 255])
        mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
        lower_red2 = np.array([160, 100, 100])
        upper_red2 = np.array([179, 255, 255])
        mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        # kombination maske

        mask = cv2.bitwise_or(mask1, mask2)
        rotallein = cv2.bitwise_and(bild2, bild2, mask=mask)
        nurrot_rgb = cv2.cvtColor(rotallein, cv2.COLOR_BGR2RGB)

        grau = cv2.cvtColor(nurrot_rgb, cv2.COLOR_BGR2GRAY)

        contours, _ = cv2.findContours(grau, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Neue leere Maske
        maskeneu = np.zeros_like(grau)
        # Flecken entfernen
        for cnt in contours:
            if cv2.contourArea(cnt) > 200:  # Threshold
                cv2.drawContours(maskeneu, [cnt], -1, 255, thickness=cv2.FILLED)
        plt.imshow(maskeneu)
        linie = cv2.Canny(maskeneu, 80, 90)  # Canny

        weiss = np.argwhere(linie == 255)  # weisse pixel
        weiss_x = weiss[:, 1]
        weiss_y = -weiss[:, 0]
        hx = max(weiss_x)
        minx = min(weiss_x)

    else:
        hx = max(weiss_x)
        minx = min(weiss_x)
    länge = hx - minx  # Länge berechnen
    # print(hx,minx)

    print(länge)  # Bis hier
    länge2 = 4.372 / länge
    print(länge2)

    # Anfang messung fisch
    print(bild.dtype)
    bild2 = bild[int(1500):int(2500), int(500):int(4750)]
    bild3 = cv2.cvtColor(bild2, cv2.COLOR_BGR2RGB)
    plt.imshow(bild3)

    # abgeänderter code von https://answers.opencv.org/question/97416/replace-a-range-of-colors-with-a-specific-color-in-python/
    #Hintergrund
    lower_blue = np.array([0, 0, 65], dtype="uint8")  # normal0,0,100 neu 65
    upper_blue = np.array([100, 100, 255], dtype="uint8")
    blue_mask = cv2.inRange(bild3, lower_blue, upper_blue)

    bild4 = bild3.copy()
    bild4[blue_mask == 255] = [255, 255, 255]

    plt.imshow(blue_mask)
    plt.figure(figsize=(10, 5), dpi=300)

    plt.subplot(1, 2, 1)
    plt.imshow(bild4)

    mask = np.zeros(bild4.shape[:2], np.uint8)  # Maske
    hinter = np.zeros((1, 65), np.float64)
    Vordergrund = np.zeros((1, 65), np.float64)
    rectangle = (10, 10, bild2.shape[1] - 10, bild4.shape[0] - 10)
    cv2.grabCut(bild4, mask, rectangle, hinter, Vordergrund, 5, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    image_segmented = bild2 * mask2[:, :, np.newaxis]


    bilddenoise = cv2.medianBlur(image_segmented, ksize=3)  # Blur
    denoisergb = cv2.cvtColor(bilddenoise, cv2.COLOR_BGR2RGB)

    # linie= cv2.Canny(denoisergb,80,90)#Canny
    linie = denoisergb

    plt.imshow(cv2.cvtColor(linie, cv2.COLOR_BGR2RGB))


    liniegrau = cv2.cvtColor(bilddenoise, cv2.COLOR_BGR2GRAY)

    contours, _ = cv2.findContours(liniegrau, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Neue leere Maske
    clean = np.zeros_like(liniegrau)

    # Grosse Flecken entfernen
    for cnt in contours:
        if cv2.contourArea(cnt) > 1000:  # Threshold
            cv2.drawContours(clean, [cnt], -1, 255, thickness=cv2.FILLED)

    plt.imshow(clean)
    linie = clean

    weiss = np.argwhere(linie == 255)  # weisse Pixel Finden


    weiss_x = weiss[:, 1]
    weiss_y = -weiss[:, 0]

    hx = max(weiss_x)
    minx = min(weiss_x)
    hy = max(-weiss_y)
    länge = hx - minx  # Länge berechnen
    längecm = länge * länge2
    print(hx, minx)
    print(länge)
    print("länge in cm", längecm)

    results.append({
        "Dateiname": os.path.basename(file),
        "Länge pixel": länge,
        "Länge cm": round(längecm, 2)
    })


#linien wurden durch chatgpt gemacht da es keinen grossen einfluss auf die ergebnisse hat
    linienbild = denoisergb.copy()
    for x in [minx, hx]:
      cv2.line(linienbild, (x, 0), (x, linienbild.shape[0]), (255, 0, 0), 3)  # Rot in RGB

      plt.figure(figsize=(12, 6))
      plt.imshow(linienbild)
      plt.title("Markierte Fischenden")
      plt.axis("off")

    titel = os.path.basename(file)
    titel2 = titel[-8:]
    print(titel2, 'nummer', img_number)

plt.show()
df = pd.DataFrame(results)
print(df)
